필요한 라이브러리 import

In [9]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import re
from konlpy.tag import Okt
import MySQLdb
import matplotlib.pyplot as plt

공지사항 데이터 추출

In [10]:
connection = MySQLdb.connect(
    host='localhost',
    user='root',
    passwd='1234',
    db='dataProject'
)
cursor = connection.cursor()

data_list_query = """
    SELECT title
    FROM notice
"""
cursor.execute(data_list_query)
notice_names = cursor.fetchall()
notice_names = [notice[0] for notice in notice_names]  # 데이터 형식에 맞게 수정
print("데이터 :", notice_names)
print("데이터 개수 :" , len(notice_names))

데이터 : ['「대학생 테마별 톡톡 멘토링」 참여 동아리 추가모집', '「2023년 국가간 청소년교류」 및 「글로벌 청소년 서밋」청소년 참가자 모집', '2023-1 실험실습 교과목 만족도 조사 안내 (드림패스 마일리지 지급)', '[강남열린대학] 인문학특강 개최 홍보', '더 안전한 경기도 영상콘텐츠 공모', '[참사람사회공헌센터] 한국장학재단 제27기 하계 대학생 재능봉사캠프 모집 안내', '[장애학생지원센터]장애인식개선교육 수강 안내', '[모두의 연구소] 2023년 AI학교 아이펠 5기 입학생 모집(~6/9)', '2023년 여름방학 대학생 교육기부 프로그램 쏙쏙캠프 참여 동아리 모집', '[교수학습혁신센터] 2023-2 혁신교수법 적용 교과목 모집 공고', '[참사람사회공헌센터] 2023 여름학기 사회봉사교과목 수강신청 안내', '동국대학교 장학 홍보대사 단체명 공모전', '[외교부] 2024-25년 대한민국 유엔 안보리 진출 제2대 청년사절 접수', '2023 어디서든 예술치유 힐링 씨어터', '장수군 대표상징물(CI) 디자인 공모전', '올담 데이터 활용 시각화 경진대회', '그린캠퍼스(게시물 관리) 시행 안내', '[5대 핵심역량 함양 프로그램] 이제호 인권 변호사와 함께하는 문화의 다양성과 존중', '2023 날씨 빅데이터 콘테스트', '기록을 바라보는 여섯 개의 시선', '제3회 해양경찰 데이터 활용 아이디어 공모전', '2023년 청년인생설계학교 2기 참여자 모집', '2023 사이버공격방어대회 참가 안내', '웹툰 창작자 발굴 및 육성을 위한 제2회 모든 웹툰 공모전', '2023년 산림공공 빅데이터 창업경진대회', '제12회 중남미 논문공모전 홍보', '2023년 「경기도 청년 노동자 통장」 참여자 모집 안내', '[5대 핵심역량 함양 프로그램] 오세훈 서울시장의 동국대 특강', '금융감독원 제 11기 대학생 금융교육 봉사단 모집', '마약류 이용 성범죄 근절 홍보 콘텐츠 공모전', '2023년 사회정책 아이디어 공모전 개최', '써라운

공지 제목 전처리 함수 정의

In [11]:
# 전처리 함수 정의
def preprocess_text(text):
    # 특수문자 제거
    text = re.sub(r'[^\w\s]', '', text)

    # 숫자 정보 제거
    text = re.sub(r'\d+', '', text)

    # 형태소 분석
    tagger = Okt()
    words = tagger.morphs(text)

    # 불용어 제거
    stop_words = ['하는', '조회', '합니다']  # 불용어 리스트
    words = [word for word in words if word not in stop_words]

    # 분석된 형태소들을 공백으로 결합하여 문장으로 반환
    text = ' '.join(words)

    return text

공지 제목 전처리 확인

In [12]:
# 전처리된 공지이름 추출
preprocessed_notice_names = [preprocess_text(notice_name) for notice_name in notice_names]

print("전처리 공지 제목", preprocessed_notice_names)

AttributeError: Java package 'kr.lucypark.okt' is not valid

word2vec 모델 학습

In [ ]:
# Word2Vec 모델 학습
sentences = [notice_name.split() for notice_name in preprocessed_notice_names]
model = Word2Vec(sentences, size=100, window=10, min_count=1, workers=4)

벡터화된 문장들 추출

In [ ]:
# 벡터화된 문장들 추출
sentence_vectors = [model.wv[sentence] for sentence in sentences]

# 문장 벡터들을 평균하여 문서 벡터 추출
document_vectors = [sum(sentence) / len(sentence) for sentence in sentence_vectors]

Elbow Method를 사용하여 적절한 클러스터 개수 k 결정

In [ ]:
inertias = []
k_values = range(1, 150)
for k in k_values:
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
    kmeans.fit(document_vectors)
    inertias.append(kmeans.inertia_)

# Elbow Method를 시각화하여 적절한 k 결정
plt.plot(k_values, inertias, 'bx-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()

k값을 결정하고 클러스터링 진행

In [ ]:
# 사용자로부터 적절한 클러스터 개수 k 입력
k = int(input("Enter the number of clusters (k): "))
# K-means 클러스터링
kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
kmeans.fit(document_vectors)

클러스터링 결과 확인

In [ ]:
cluster_labels = kmeans.labels_
for i, notice_name in enumerate(notice_names):
    print(f"{notice_name} - Cluster: {cluster_labels[i]}")

사용자가 직접 라벨 지정

In [ ]:
# 클러스터 묶기
cluster_groups = {}
for i, notice_name in enumerate(notice_names):
    cluster = cluster_labels[i]
    if cluster in cluster_groups:
        cluster_groups[cluster].append(notice_name)
    else:
        cluster_groups[cluster] = [notice_name]

# 그룹 이름 결정
group_names = {}
for cluster, group in cluster_groups.items():
    print(f"Cluster {cluster}:")
    print(group)
    group_name = input("Enter the name for this group: ")
    group_names[cluster] = group_name

# 그룹 이름 출력
print("Group Names:")
for cluster, group_name in group_names.items():
    print(f"Cluster {cluster}: {group_name}")